In [1]:

import numpy as np
import pandas as pd
import pickle
import networkx as nx 
import osmnx as ox 
import numpy as np 
import seaborn as sns 

In [ ]:
#Load volume dataset
# csv_pd=pd.read_excel(r"C:\Users\anton\Desktop\Master-Oppgave\Kode\New_Traffic\weekly_data\weekly_data_xl.xlsx")

# # Load graph
# with open(r"C:\Users\anton\Desktop\Master-Oppgave\Kode\New_Traffic\data\BaseGraph_E_NOR_wagrades.pickle", 'rb') as file:
#     G = pickle.load(file)


In [17]:
import pandas as pd
import networkx as nx
from shapely.geometry import Point, LineString
import pickle
from tqdm import tqdm 

# read in the road network graph
with open(r"C:\Users\anton\Desktop\Master-Oppgave\Kode\New_Traffic\data\BaseGraph_E_NOR_wagrades.pickle", 'rb') as file:
    G = pickle.load(file)

# create a dictionary mapping node IDs to (x, y) coordinates
node_coords = {node: (data['x'], data['y']) for node, data in G.nodes(data=True)}

# read in the dataframe with the traffic points
points_df = pd.read_excel(r"C:\Users\anton\Desktop\Master-Oppgave\Kode\New_Traffic\weekly_data\weekly_data_xl.xlsx")
#Remove volume1 and volume2
points_df=points_df.drop("volume1", axis="columns")
points_df=points_df.drop("volume2", axis="columns")

# convert latitude and longitude columns to shapely Point objects
points_df['geometry'] = points_df.apply(lambda x: Point(x['lon'], x['lat']), axis=1)

# loop through each point in the dataframe and find the nearest edge on the road network graph
for idx, row in tqdm(points_df.iterrows(), total=points_df.shape[0]):
# get the shapely Point object for the current traffic point
    point = row['geometry']
# find the nearest point on the road network graph
    nearest_node = None
    min_distance = float('inf')
    for node in G.nodes:
        # extract (x, y) coordinates from node attributes
        x, y = G.nodes[node]['x'], G.nodes[node]['y']
        node_point = Point(x, y)
        distance = point.distance(node_point)
        if distance < min_distance:
            nearest_node = node
            min_distance = distance
    # find the closest edge to the nearest point on the road network graph
    nearest_edge = None
    min_distance = float('inf')

    for edge in G.edges:
    # get the coordinates of the nodes in the edge tuple
        node1 = G.nodes[edge[0]]
        node2 = G.nodes[edge[1]]
        coords = [(node1['x'], node1['y']), (node2['x'], node2['y'])]

        # construct a LineString from the coordinates and compute the distance to the point
        distance = point.distance(LineString(coords))

        if distance < min_distance:
            nearest_edge = edge
            min_distance = distance
    # if the distance between the point and the closest point on the edge is below a certain threshold,
    # add the volume data to that edge's attribute in the road network graph
    if min_distance < 50: # adjust the threshold as needed
    # create a dictionary to hold the edge attributes
        edge_attrs = {}

        # update the dictionary with the new 'volume' attribute
        edge_attrs['volume'] = row['volume']

        # update the edge attributes in the graph using the dictionary
        G.edges[nearest_edge].update(edge_attrs)


C:\Users\anton\AppData\Local\Temp\ipykernel_21128\3093859715.py:9: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  G = pickle.load(file)
  0%|          | 0/7273 [00:00<?, ?it/s]C:\Users\anton\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\shapely\measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
100%|██████████| 7273/7273 [30:32<00:00,  3.97it/s]


In [18]:
volume_testing=nx.to_pandas_edgelist(G)

volume_testing.to_csv("network_with_volume.csv", index=False)